In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
# with implementation from http://mnemstudio.org/path-finding-q-learning-tutorial.htm

In [2]:
#use trainning data to build the recommendation
data=pd.read_csv('/Users/irwang/Desktop/DMUU/final_labeled_data_train.csv')
data.head()

,user_id,s,a,sp,r,movie_num
0,133,12,1,11,-3,1
1,133,12,2,11,-2,1
2,133,12,3,11,-3,1
3,133,12,4,11,-3,1
4,133,12,5,11,-3,1


In [3]:
test=pd.read_csv('/Users/irwang/Desktop/DMUU/final_labeled_data_test.csv')
test.head()
test_u=test[test['r']>0][['user_id','s','sp','r']].drop_duplicates()

In [4]:
#make it consistent with index
data['s']=data['s']-1
data['a']=data['a']-1
data['sp']=data['sp']-1
data.head()
avg_r=data.groupby(['s', 'a']).r.agg(['mean']).reset_index()
avg_r[avg_r['mean']>0]

,s,a,mean
15,1,1,0.545455


In [6]:
def update_q(s,sp,a,alpha,gamma):
    rsa = r[s, a]
    qsa = q[s, a]
    new_q = qsa + alpha * (rsa + gamma * max(q[sp, :]) - qsa)
    q[s, a] = new_q
    return r[s, a]

In [5]:
def learning(alpha,gamma):
    rewards=[]
    epsilon = 1.0 
    for e in range(int(n_episodes)):
        #print e
        states = np.arange(1,14)
        random_state.shuffle(states)
        current_state = states[0]
        total_rewards = 0
        for i in range(steps):
            if np.random.rand(1)[0] >= epsilon:
                action = np.argmax(q[current_state])
            else:
                action = np.random.randint(0,14)
            next_state=int(data[(data['s']==current_state)&(data['a']==action)].sample()['sp'])
            reward = update_q(current_state, next_state, action,alpha=alpha, gamma=gamma) 
            total_rewards += reward
            current_state = next_state
        epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*n_episodes) 
        rewards.append(total_rewards)
    return rewards

In [7]:
r = avg_r.pivot(index='s', columns='a', values='mean').as_matrix()
n_episodes = 200
max_epsilon = 1
min_epsilon = 0.05
decay_rate = 0.005  
random_state = np.random.RandomState(1999)
steps=100    

In [8]:
for al in np.arange(0.05,0.55,0.05):
    # defines the reward/connection graph
    q = np.full(r.shape, -100)
    rewards_list=learning (alpha=al,gamma=0.99)
    rec=[]
    for i in range(14):
        d={}
        d['s']=i+1
        d['recommend']=np.argmax(q[i])+1
        rec.append(d.copy())
    recommender=pd.DataFrame(rec)
    pred=pd.merge(test_u,recommender)
    print al,len(pred[pred['sp']==pred['recommend']])/float(len(pred)),len(pred[(pred['sp']==pred['recommend'])&(pred['r']>3)])/float(len(pred[pred['r']>3]))

0.05 0.16041326808 0.168604651163
0.1 0.196846112017 0.201550387597
0.15 0.202283849918 0.212209302326
0.2 0.201196302338 0.210271317829
0.25 0.21098423056 0.218992248062
0.3 0.200652528548 0.203488372093
0.35 0.200108754758 0.210271317829
0.4 0.203915171289 0.213178294574
0.45 0.205002718869 0.21996124031
0.5 0.195214790647 0.202519379845


In [10]:
#SARSA

In [18]:
def update_q_ss(s,sp,a,ap,alpha,gamma):
    rsa = r[s, a]
    qsa = q_ss[s, a]
    new_q = qsa + alpha * (rsa + gamma * (q_ss[sp,ap]) - qsa)
    q_ss[s, a] = new_q
    return r[s, a]

In [22]:
def learning_ss(alpha,gamma):
    rewards=[]
    epsilon = 1.0 
    for e in range(int(n_episodes)):
        #print e
        states = np.arange(1,14)
        random_state.shuffle(states)
        current_state = states[0]
        total_rewards = 0
        for i in range(steps):
            if np.random.rand(1)[0] >= epsilon:
                action = np.argmax(q[current_state])
            else:
                action = np.random.randint(0,14)
                print action
            next_state=int(data[(data['s']==current_state)&(data['a']==action)].sample()['sp'])
            next_action=np.random.randint(0,14)
            reward = update_q_ss(current_state, next_state, action,next_action,alpha=alpha, gamma=gamma) 
            total_rewards += reward
            current_state = next_state
        epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*n_episodes) 
        rewards.append(total_rewards)
    return rewards

In [23]:
for al in np.arange(0.05,0.55,0.05):
    # defines the reward/connection graph
    q_ss = np.full(r.shape, -100)
    rewards_list_ss=learning_ss(alpha=al,gamma=0.99)
    rec_ss=[]
    for i in range(14):
        d_ss={}
        d_ss['s']=i+1
        d_ss['recommend']=np.argmax(q[i])+1
        rec_ss.append(d.copy())
    recommender_ss=pd.DataFrame(rec_ss)
    pred_ss=pd.merge(test_u,recommender_ss)
    print al,len(pred_ss[pred_ss['sp']==pred_ss['recommend']])/float(len(pred_ss)),len(pred_ss[(pred_ss['sp']==pred_ss['recommend'])&(pred_ss['r']>3)])/float(len(pred_ss[pred_ss['r']>3]))

12
10
3
3
10
3
6
8
13
5
5
5
13
8
8
5
13
8
0
5
0
10
6
4
6
4
1
3
12
13
10
4
5
10
10
6
3
2
8
13
12
12
1
12
13
10
8
8
4
5
8
8
7
13
8
5
7
2
3
1
9
2
9
1
6
5
5
8
12
1
6
12
0
12
11
1
4
6
3
9
11
3
5
0
1
6
2
12
13
5
5
10
12
8
2
10
7
3
1
8
11
9
9
3
1
2
6
3
0
5
3
1
11
10
9
3
10
4
11
7
8
13
9
10
12
11
0
4
9
12
2
8
4
9
11
3
11
6
9
9
12
8
7
10
9
1
2
10
9
7
1
3
8
13
12
7
12
9
11
7
11
4
9
10
3
8
6
11
8
6
1
3
2
3
0
2
7
0
6
5
12
0
12
5
13
2
9
5
4
5
10
4
6
3
3
12
3
6
4
13
13
3
4
12
5
3
9
13
4
7
12
1
2
7
9
8
1
10
7
11
8
3
9
11
7
9
3
11
5
13
11
10
7
11
10
5
13
10
8
11
8
0
10
12
0
8
10
3
11
2
7
2
8
1
4
8
1
12
5
4
2
4
2
6
10
7
2
2
5
5
2
12
10
5
0
4
4
5
9
5
4
12
10
9
5
2
13
4
4
1
6
3
2
1
3
2
8
9
3
5
12
0
4
11
1
5
13
2
4
8
5
7
5
4
3
5
4
8
6
6
2
8
8
1
11
8
3
13
5
10
8
5
5
10
0
11
4
7
11
1
4
8
7
7
10
2
3
13
7
2
9
1
9
1
1
12
6
5
6
3
7
12
2
1
3
9
10
1
2
0
6
6
3
7
0
13
1
11
0
10
2
3
9
9
3
8
6
11
11
8
8
2
2
6
8
7
4
4
10
11
1
7
7
10
1
6
7
10
3
11
13
5
9
7
12
1
4
8
4
0
12
5
4
6
13
0
7
6
9
6
5
12
8
1
7
10
9
4
1
0
3
10
8

1
12
12
8
5
2
12
9
4
8
11
2
4
2
9
13
12
11
13
13
11
10
10
7
2
0
5
4
4
12
5
1
8
11
5
8
1
1
10
13
0
8
0
2
2
9
7
3
12
7
7
5
1
1
4
11
7
11
1
7
0
2
0
12
12
2
5
5
10
10
2
6
0
7
5
6
7
0
9
0
10
2
0
11
13
11
11
13
1
0
1
0
12
10
3
13
1
3
8
2
9
13
9
13
4
4
1
11
6
0
1
11
8
8
8
12
12
9
10
4
2
10
13
4
2
9
12
11
5
6
4
13
0
7
13
5
11
5
8
11
1
0
8
11
5
8
9
13
10
6
13
0
8
7
2
12
1
6
7
7
10
6
4
12
3
10
10
11
1
4
0
11
11
12
8
11
2
9
9
9
0
0
5
4
0
9
11
0
6
10
12
13
3
0
3
0
0
1
0
6
10
3
3
2
10
3
11
10
12
1
5
5
8
2
4
12
4
7
13
12
8
2
8
2
7
7
5
8
8
5
0
1
10
12
1
4
10
1
13
6
5
2
10
8
6
8
7
11
1
12
1
8
13
5
9
9
11
10
3
11
7
11
7
7
9
3
13
4
0
9
6
5
8
9
13
7
4
12
4
3
13
9
9
3
12
1
5
11
2
4
12
7
11
8
2
9
3
10
4
4
12
0
9
0
12
8
11
2
8
6
12
2
4
4
5
1
12
2
3
10
4
6
8
3
13
5
4
11
8
12
5
13
3
11
1
4
11
1
9
2
2
12
11
10
0
11
5
4
10
5
3
3
12
0
4
3
8
0
9
2
11
6
4
2
6
3
13
10
7
11
7
6
5
0
5
8
2
5
7
8
11
5
0
2
6
3
5
2
8
10
0
6
3
2
10
8
2
5
11
5
0
7
11
0
1
7
1
13
1
5
4
2
4
7
5
3
4
9
4
9
4
3
6
2
12
7
0
7
3
11
5
12
7
11
11
7
1

4
1
3
10
0
5
6
9
0
7
13
5
7
10
6
13
12
7
7
0
3
4
13
0
12
9
9
13
1
9
4
7
3
5
12
13
7
7
0
11
4
4
13
2
7
5
7
13
7
6
5
7
5
2
5
2
8
13
5
2
5
7
1
7
6
2
7
0
12
11
3
13
2
5
0
9
10
7
6
4
5
13
8
11
10
1
0
11
10
3
0
11
10
1
10
3
1
11
2
9
10
6
1
9
7
11
9
3
8
6
10
7
13
4
2
9
12
5
11
1
8
2
6
4
4
12
4
13
9
7
5
0
9
2
1
5
9
10
2
1
11
0
10
12
6
10
13
5
10
7
9
2
12
13
12
5
1
4
1
3
5
8
6
8
6
0
9
8
13
1
11
7
11
2
7
12
7
9
7
13
2
9
4
6
11
3
1
6
7
9
2
2
7
11
3
10
6
4
9
2
0
11
7
4
7
7
13
6
7
7
5
4
13
12
12
2
3
5
0
5
9
11
4
12
1
6
5
12
2
1
5
2
2
3
4
11
5
8
2
11
2
13
1
13
3
4
8
6
4
4
5
7
0
11
10
11
9
12
5
9
0
10
2
4
13
0
0
5
0
12
5
4
8
7
12
13
3
10
2
7
0
5
13
11
4
5
13
7
6
3
0
10
5
0
0
13
11
3
9
5
8
7
5
11
6
10
12
9
10
10
11
4
11
4
12
11
0
11
12
5
7
1
1
2
9
8
2
6
1
3
1
4
3
10
9
4
7
7
3
3
12
8
6
12
1
4
13
11
2
8
10
1
12
10
9
7
9
9
0
6
3
11
5
5
7
0
5
6
7
13
9
3
13
1
12
11
12
2
8
11
10
2
0
7
4
6
7
8
10
3
10
12
1
3
4
7
2
9
11
2
3
11
3
0
5
3
10
2
12
8
8
5
13
2
4
2
10
9
11
10
5
7
0
1
9
10
4
11
13
1
0
3
12
7
13
5
3
13

1
6
13
4
10
2
10
0
7
6
5
11
10
12
10
5
0
12
11
8
13
2
9
7
7
11
2
8
7
8
6
12
8
7
7
2
12
10
1
4
3
11
5
8
11
1
4
5
1
11
10
3
7
2
13
11
3
9
4
11
0
4
13
8
10
6
0
8
13
12
10
1
11
2
3
3
10
13
9
4
8
12
2
6
1
11
7
11
1
8
5
1
6
12
1
10
5
7
6
5
8
13
1
13
13
7
3
12
9
6
4
11
6
1
6
5
3
11
9
1
0
12
6
1
5
9
11
7
11
9
5
3
13
4
9
5
7
13
10
5
7
5
1
10
4
7
11
1
7
4
3
11
5
13
9
3
12
5
5
0
10
3
9
9
12
1
13
10
3
11
4
13
8
6
1
2
2
4
8
8
9
5
8
7
5
3
6
12
8
13
9
9
4
5
10
13
13
7
2
8
4
11
10
7
3
9
1
7
9
0
2
4
8
2
10
9
5
1
12
4
11
0
8
6
10
0
7
7
0
11
11
5
5
9
6
0
8
9
6
5
3
3
1
0
10
6
7
2
10
6
3
0
12
2
5
13
6
12
2
7
0
6
4
8
10
6
2
8
0
5
10
12
11
8
6
11
5
6
12
10
4
11
13
13
3
10
3
13
13
4
0
0
8
13
8
2
5
3
8
9
11
10
5
11
10
4
11
10
4
0
2
9
9
10
10
4
8
0
2
3
1
1
11
4
2
2
4
6
8
1
6
5
2
6
0
10
12
10
4
0
11
6
7
12
12
3
10
8
5
4
7
9
12
13
2
5
1
3
4
12
11
4
13
11
13
7
1
7
12
4
6
8
4
9
2
0
7
11
6
3
9
6
10
9
3
2
11
2
2
1
0
7
5
6
6
1
7
12
2
9
7
4
0
2
4
9
6
0
1
12
3
3
2
7
11
0
9
5
6
10
1
10
0
4
8
4
13
4
8
6
5
0
6
11
10
12
13


KeyboardInterrupt: 